In [1]:
from collections import defaultdict
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import re
import numpy as np

In [2]:
def parseData(filePath):
    data = []
    listOfSentences = []
    
    with open(filePath, 'r') as file:
        sentId = ""
        
        for line in file:
            line = line.strip()
            if line.startswith("# sent_id"):
                newLine = line.split("=")
                newSentence = newLine[-1].strip()
                sentId = newSentence
            
            elif line.startswith("# text"):
                newLine = line.split("=")
                newSentence = newLine[-1].strip()
                listOfSentences.append(newSentence)
                
                
            elif line and not line.startswith("#"):
                divisions = line.split()
                if len(divisions) >= 5:
                    data.append([sentId, divisions[0], divisions[1], divisions[3]])
                    
                    
    return data, listOfSentences


data, trainSentences = parseData("./NLP2/train.txt")

In [3]:
def calculateTagProbs(data):
    tagsAfterTagsCounts = defaultdict(lambda : defaultdict(int))
    tagCounts = defaultdict(int)
    tagProbs = defaultdict(lambda : defaultdict(float))
    
    for i in range(len(data)-1):
        currentTag = data[i][3]
        nextTag = data[i+1][3]
        tagsAfterTagsCounts[currentTag][nextTag] += 1
        tagCounts[currentTag] += 1
        
    for current, nextDict in tagsAfterTagsCounts.items():
        for nextT, count in nextDict.items():
            tagProbs[current][nextT] = count / tagCounts[current]
    
    return tagProbs

def calculateWordTagProbs(data):
    vocabularySet = set([])
    wordTagCounts = defaultdict(lambda: defaultdict(int))
    wordTagProbs = defaultdict(lambda: defaultdict(float))
    tagCounts = defaultdict(int)
    wordCounts = defaultdict(int)
    
    for i in range(len(data)):
        tag = data[i][3]
        word = data[i][2]
        wordTagCounts[tag][word] += 1
        tagCounts[tag]+=1
        vocabularySet.add(word)
    
    tagCnts = len(tagCounts)
        
    for tag, words in wordTagCounts.items():
        for word, count in words.items():
            wordTagProbs[tag][word] = (count + 1) / tagCounts[tag]
    
    return wordTagProbs, tagCnts, vocabularySet, wordTagCounts, tagCounts

In [4]:
def viterbi(sentence, tagProbs, wordTagProbs):
    v = [{1: "a"}]
    pathFollowed = {}
    
    # Base Case:
    for tag in wordTagProbs.keys():
        v[0][tag] = (1/tagCnts)*wordTagProbs[tag][sentence[0]]
        pathFollowed[tag] =[tag]
    
    # Recursive case
    for t in range(1, len(sentence)):
        v.append({})
        newpath = {}

        for tag in wordTagProbs.keys():
            (prob, state) = max((v[t-1][prevTag] * tagProbs[prevTag][tag] * wordTagProbs[tag][sentence[t]], prevTag)
                                for prevTag in wordTagProbs.keys())
            v[t][tag] = prob
            newpath[tag] = pathFollowed[state] + [tag]

        pathFollowed = newpath
    
    
    n = len(sentence) - 1
    (prob, state) = max((v[n][tag], tag) for tag in wordTagProbs.keys())
    
    return [prob, pathFollowed[state]]

In [5]:
# We need to perform add one smoothing on the training as well as the test data
# No need to perform smoothing on the tag probs
sentence = ["I", "am", "a", "boy", "."]
wordTagProbs, tagCnts, vocabularySet, x, y = calculateWordTagProbs(data)
tagProbs = calculateTagProbs(data)
viterbi(sentence, tagProbs, wordTagProbs)[1]

['PRP', 'VBP', 'DT', 'NN', '.']

In [14]:
def getCorrespondingTags(data):
    dictData = {}
    for smallArr in data:
        key = smallArr[0]
        value = smallArr[3]
        if key in dictData:
            dictData[key].append(value)
        else:
            dictData[key] = [value]

    result = []
    for key, values in dictData.items():
        result.append(values)

    return result

def getCorrespondingSentence(data):
    dictData = {}
    for smallArr in data:
        key = smallArr[0]
        value = smallArr[2]
        if key in dictData:
            dictData[key].append(value)
        else:
            dictData[key] = [value]

    result = []
    for key, values in dictData.items():
        result.append(values)

    return result

def getSmoothingCount(data):
    smoothingCount = 0
    for i in range(len(data)):
        if wordTagProbs[data[i][3]][data[i][2]] == 0:
            smoothingCount+=1
    return smoothingCount

testData, _ = parseData("./NLP2/test.txt")
getTags = getCorrespondingTags(testData)
smoothingCount = getSmoothingCount(testData)
sentences = getCorrespondingSentence(testData)
print(getTags)

[['DT', 'NN', 'IN', 'NN', 'IN', 'JJ', 'NNS', 'IN', 'JJ', 'NNP', ':'], ['NNS', 'IN', 'DT', 'RB', 'JJ', 'NN', 'IN', 'NNS'], ['NN', '.'], ['JJ', 'NNS', 'IN', 'NN', 'CC', 'NN', 'VBP', 'VBN', 'DT', 'NN', 'IN', 'JJ', 'JJ', 'NN', 'NN', '.', '-LRB-', 'CD', 'SYM', 'CD', '-RRB-'], ['NNS', 'VBP', 'VBN', 'DT', 'JJ', 'NNS', 'IN', 'NN', '-LRB-', 'CD', 'SYM', 'CD', '-RRB-', ':'], ['NNS', 'VBP', 'VBN', 'DT', 'JJ', 'NNS', 'WDT', 'VBP', 'JJ', 'CC', 'JJ', 'NNS', '-LRB-', 'CD', '-RRB-', ':'], ['NNS', 'VBP', 'VBN', 'DT', 'JJ', 'NNS', 'IN', 'NN', '-LRB-', 'CD', 'SYM', 'CD', '-RRB-', ':'], ['CC', 'JJ', 'NNS', 'VBP', 'VBN', 'DT', 'JJ', 'NNS', 'IN', 'IN', 'HYPH', 'NN', 'SYM', 'IN', 'HYPH', 'NN', 'NNS', 'VBD', 'IN', 'DT', 'NN', 'IN', 'PRP$', 'NN', '.', '-LRB-', 'CD', '-RRB-'], ['IN', 'JJ', 'NNS', ',', 'NNS', 'RB', 'VBP', 'DT', 'NN', 'IN', 'NN', 'VBG', 'DT', 'NNS', 'CC', 'NNS', 'IN', 'NN', 'CC', 'NN', '.', '-LRB-', 'CD', '-RRB-'], ['WP', 'RB', 'MD', 'RB', 'VB', 'VBN', 'IN', 'NN', 'IN', 'NN', 'VBZ', 'DT', 'VBG', 

In [12]:
# Perform Smoothing
def smoothenTransitionProbability(data, tagProbs, wordTagProbs, countTags, countWords):
    v = len(vocabularySet)
    for i in range(len(data)):
        wordTagProbs[data[i][3]][data[i][2]] = (countWords[data[i][3]][data[i][2]] + 1) / (v + countTags[data[i][3]])
    
    for tag, words in wordTagProbs.items():
        for word, prob in words.items():
            wordTagProbs[tag][word] = (countWords[tag][word] + 1) / (v + countTags[tag])
    
    return wordTagProbs

wordTagProbs = smoothenTransitionProbability(testData, tagProbs, wordTagProbs, y, x)

In [13]:
results = []
for i in range(len(sentences)):
    sentence = sentences[i]
    result = viterbi(sentence, tagProbs, wordTagProbs)[1]
    results.append(result)

# Flatten the lists
A_flat = [item for sublist in getTags for item in sublist]
B_flat = [item for sublist in results for item in sublist]

# Calculate metrics
accuracy = accuracy_score(A_flat, B_flat)
precision = precision_score(A_flat, B_flat, average='weighted', zero_division=1)
recall = recall_score(A_flat, B_flat, average='weighted', zero_division=1)
f1 = f1_score(A_flat, B_flat, average='weighted')

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

Accuracy: 0.9572074526040532
Precision: 0.9575142654288404
Recall: 0.9572074526040532
F1 Score: 0.9561591969387586


## Generating a TSV File according to the format specified in the Problem Statement